In [49]:
def compara(ai,aj,max=True):
  # Higher is better.
  if (max):
    res = 0
    if (ai > aj):
      res = 1
  # Lower is better.    
  else:
    res = 1
    if (ai > aj):
      res = 0

  return res  

# Cálculo de phi+ para 1 critério (coluna).
def tabelaC(df,crit,min):
  tc1 = []
  for i in df.index:
    ai = df.loc[i,crit]
    aux = []
    # Comparação de ai com vários aj.
    for j in df.index:
      aj = df.loc[j,crit]
      if (min):  
        res = compara(ai,aj,False)
      else:
        res = compara(ai,aj,True)  
      aux.append(res)
    tc1.append(aux)

  return tc1


# Cálculo de phi- para 1 critério (coluna).
def tabelaCM(df,crit,min):
  tc1 = []
  for i in df.index:
    ai = df.loc[i,crit]
    aux = []
    # Comparação de ai com vários aj.
    for j in df.index:
      aj = df.loc[j,crit]
      if (min):  
        res = compara(aj,ai,False)
      else:
        res = compara(aj,ai,True)  
      aux.append(res)
    tc1.append(aux)

  return tc1

# Cálculo do Phi+ para todos os critérios (colunas).
def calcPhipPesos(df,pesos,objetivos):

  # Obtendo o número de alternativas (linhas da tabela).
  rows = df.index
  m = len(rows)
  # Obtendo os critérios (colunas da tabela).
  columns = df.columns
  n = len(columns) 

  #-----------------------------------------------------------
  # Cálculo do Phi+ (todas as colunas).
  #-----------------------------------------------------------
  # Comparando as alternativas em relação ao primeiro critério.
  tc = tabelaC(df,columns[0],objetivos[0]) 
  dc = pesos[0]*pd.DataFrame(data=tc)
  #print("Cálculo do critério "+columns[0]) 
 
  # Comparando as alternativas nos demais critérios e armazenado
  # o resultado acumulado em dc.
  for i in range(1,n):
    tci = tabelaC(df,columns[i],objetivos[i])
    dci = pesos[i]*pd.DataFrame(data=tci)
    #print("Cálculo do critério "+columns[i])
    dc = dc.add(dci, fill_value=0)

  # Resultando final: para cada linha soma das colunas.
  dc.loc[:,'Total'] = dc.sum(axis = 1, skipna = True)
  # Dividir por 1/(m-1) a soma total.
  dc['Total'] = (dc['Total']/(m-1)).round(2)
  #-----------------------------------------------------------

  return dc['Total']


# Cálculo do Phi- para todos os critérios (colunas).
def calcPhinPesos(df,pesos,objetivos):

  # Obtendo o número de alternativas (linhas da tabela).
  rows = df.index
  m = len(rows)
  # Obtendo os critérios (colunas da tabela).
  columns = df.columns
  n = len(columns) 

  #-----------------------------------------------------------
  # Cálculo do Phi- (todas as colunas).
  #-----------------------------------------------------------
  # Comparando as alternativas em relação ao primeiro critério.
  tc2 = tabelaCM(df,columns[0],objetivos[0]) 
  dc2 = pesos[0]*pd.DataFrame(data=tc2)
  #print("Cálculo do critério "+columns[0]) 
 
  # Comparando as alternativas nos demais critérios e armazenado
  # o resultado acumulado em dc.
  for i in range(1,n):
    tci2 = tabelaCM(df,columns[i],objetivos[i])
    dci2 = pesos[i]*pd.DataFrame(data=tci2)
    #print("Cálculo do critério "+columns[i])
    dc2 = dc2.add(dci2, fill_value=0)

  # Resultando final: para cada linha soma das colunas.
  dc2.loc[:,'Total'] = dc2.sum(axis = 1, skipna = True)
  # Dividir por 1/(m-1) a soma total.
  dc2['Total'] = (dc2['Total']/(m-1)).round(2)

  return dc2['Total']



# Cálculo do índice de ranqueamento do Promethee II.
def prometheeIIpesos(df,pesos,objetivos):
  #print("Iniciando cálculos do Promethee II...")  

  # Cálculo do Phi+.
  #print('Cálculos do Phi+...')
  dcp = calcPhipPesos(df,pesos,objetivos)
  
  # Cálculo do Phi-.
  #print('Cálculos do Phi-...')
  dcn = calcPhinPesos(df,pesos,objetivos)

  #print('Cálculos do Phi = Phi+ - Phi-...')

  return dcp-dcn

In [65]:
import pandas as pd

pd.set_option('display.max_rows', None)

pesos = [0.2, 0.3, 0.5, 0.3, 0.3, 0.4, 0.2, 0.4, 0.4, 0.5, 0.5, 1]

objetivos = [False, False, False, False, False, False, False, False, False, False, False, False]

df = pd.read_csv('subdim.csv', index_col=0, header=0)

In [66]:
phi = prometheeIIpesos(df,pesos,objetivos)
df.loc[:,'Phi']= list(phi)
df

,Densidade Empresarial,Renda,Educação,Valores Solidários - programas e ações,Tecido Social,Tecido Empresarial,Articulação,Gestão Pública,Participação e Controle Social,Vantagens Locais,Organização Produtiva,Comércio Internacional,Phi
Adamantina,0.292112,0.709371,0.844012,0.571429,0.003483,0.000000,0.500000,0.695181,0.404762,0.439765,0.491475,0.067792,1.59
Adolfo,0.294667,0.623339,0.801920,0.000000,0.000657,0.000000,0.000000,0.533664,0.374048,0.413952,0.352418,0.067792,-0.83
Aguaí,0.282476,0.594759,0.771491,0.000000,0.001220,0.002002,0.000000,0.326012,0.400238,0.344832,0.500990,0.067792,-0.75
Águas da Prata,0.270997,0.671739,0.826817,0.000000,0.000258,0.000000,0.083333,0.787747,0.367143,0.424545,0.474748,0.067792,0.49
Águas de Lindóia,0.435084,0.647601,0.791001,0.000000,0.000610,0.000200,0.250000,0.662081,0.362302,0.400696,0.461352,0.067792,0.28
Águas de Santa Bárbara,0.388350,0.654867,0.819654,0.000000,0.000023,0.000000,0.750000,0.642609,0.542143,0.398232,0.419054,0.067792,0.26
Águas de São Pedro,0.586234,0.876034,0.939773,0.571429,0.000985,0.000000,0.166667,0.699065,0.417143,0.613215,0.446190,0.067792,1.41
Agudos,0.309981,0.598214,0.828578,0.571429,0.002791,0.000000,0.000000,0.570823,0.463016,0.399806,0.494136,0.093421,1.43
Alambari,0.206951,0.569713,0.762389,0.000000,0.000281,0.000000,0.000000,0.786732,0.311270,0.320721,0.474018,0.067792,-1.57
Alfredo Marcondes,0.228440,0.589000,0.782711,0.000000,0.000035,0.000000,0.083333,0.449275,0.324206,0.399105,0.334394,0.096971,-0.93


In [69]:
df_sorted = df.sort_values(by='Phi', ascending=False)
df_sorted["Phi"]

Santos                        4.56
Sorocaba                      4.47
Campinas                      4.44
Barueri                       4.33
São Caetano do Sul            4.32
São Bernardo do Campo         4.31
São Paulo                     4.28
Santo André                   4.27
Piracicaba                    4.25
Limeira                       4.17
Americana                     4.12
Jundiaí                       4.12
Bauru                         4.10
São José dos Campos           4.06
Rio Claro                     4.00
Valinhos                      4.00
São José do Rio Preto         3.99
Santana de Parnaíba           3.97
Marília                       3.93
Vinhedo                       3.92
Ribeirão Preto                3.86
Franca                        3.85
Amparo                        3.76
Indaiatuba                    3.76
Araraquara                    3.69
Jaú                           3.68
Barretos                      3.66
São Carlos                    3.58
Araçatuba           